In [ ]:
# ResNet
# 
# vanishind and exploding gradient : 
# ağırlıklar ağın çıktısından itibaren güncellene güncellene gidiyor, zincirleme kuralı
# belli bir süre sonra türevler sıfırlanmaya başlıyor ve ağın girişindeki(çıkış tarafına yakın olan) güncellenirken giriş tarafına yakın olanlar güncellenmiyor
# yani girdi katmanına yakın olan ağırlıklar eğitilmiyor.
# buna nasıl bir çözüm üretiyoruz.
# residual (artık) öprenme yapısı: 
# modelin aslında daha önce öğrenmiş olduğu bilgileri bir sonraki mödüle aktarılması tarzı bağlantı kuruyorlar.
# feature mapleri sonrakine aktarıyoruz, kaybolan bişi varsa kaybetmiyoruz.

# 56 katmanlı 20 katmanlıya göre daha çok error oranı verir, bunun bir optimizasyon hatası olduğunu söylemişler.
# çözüm : bir artık haritalama yöntemi kullanmak, bunlara skip connections diyorlar
# temel mantık belli bir katmandaki aktivasyonu başka bir katmana beslemek
# bu da daha derin bir ağ elde etmiş oluyoruz ve gradiantların yok olması probleminden kurtulmuş oluyoruz.

# Residual Block (Artık blok):
# girdiye vermiş olduğumuz x'i conv çıktısına ekliyoruz.
# iki yapı var 
# Add: çıkan bir feature map'i diğer maple birebir toplanıyor, boyut değişmez! ve bir tane feature map elde ediyoruz !
# Concentinate : filtrelerden çıkan mapler yan yana gelip diziliyordu bu birleştirme yapıyoruz ve birden fazla feature map elde ediyoruz !

# Her bir blok 2 tane 3x3 conv'dan oluşuyor.
# sadece 2 tane pooling var
# peki görüntü nasıl küçülüyor : conv katmanlarındaki filtrelerin stride'nı 2 yaparak küçültmüşler.
# tam bağımlı katmanlar kullanılmıyor, 1000lik çıktı.

# ResNet'in getirdiği yenilik sayesinde daha derin modellerin daha performanslı sonuç vermesi ve artık eğitilebilir olması ...

# En iyi CNN mimarisi, büyük bir yenilik, insandan bile daha iyi performans göstermiş...
# daha sonra bazi mimarilerin güçlü yerleri birleştirildi daha iyi modeller çıktı..

# ResNet'te iki farklı block var: 
# Identıty Block:
# -Conv İşlem
# -Batch Normalizasyon : Çarpma ve toplama işlemi sonucunda yeni oluşturulan feature mapler eğer büyürse, sayılar yüksek değere gelirse bunları küçültüyor norm. yapıyor.
# -Relu
# -Conv
# -Batch Norm
# -Topla
# -Relu

# Convolutional Block
# Yaptığımız conv işlemleri sonucunda bi miktar feature maplerde küçülme meydana gelebiliyor
# eğer x ile bu feature map'i birleştirmeye kalkarsak hata verecektir.
# bunun için x'e conv uygulayıp aynı boyutlara getirip add yapıyoruz.


# ÖDEV ARAŞTIR
# GlobalAveraging2D ile Flatten arasındaki farkı sınavda soruyum!!!.
# GlobalAveragePooling : parametre sayısının düşüşünde çok önemli rol oynar. GAP2D yaparsak sadece 2048'lik hale getiriyoruz grüntüyü.
# Flatten : Flat yaparsak 7x7x2048 bir vektör hale getiriyoruz görüntüyü
# burda 49x2048'lik feature azalması var bu da bir sonraki katmana olan bağlantı sayısını azaltır ve ağırlık sayısını düşürür
# GlobalAveragePooling: tüm özellik haritalarındaki bir tane sayı alıyor (ortalamasını alıp)
# Flatten: bütün sayıları alıyor


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras


from keras.utils import np_utils
from tensorflow.keras import regularizers
from keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization,Activation,Add,Input
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D ,Input,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.layers import Concatenate
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD

In [ ]:
def identity_block(X,f,filters,stage,block):

  conv_name_base='res'+str(stage)+ block+'_branch'  #hangi stage'in hangi bloğuna ait
  bn_name_base='bn'+str(stage)+ block+'_branch'
  F1,F2,F3=filters

  X_shortcut=X    # ilk giren X buraya aktarılıyor sonradan kesip eklemek için

  X=Conv2D(filters=F1,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2a')(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
  X=Activation('relu')(X)

  X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name_base+'2b')(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
  X=Activation('relu')(X)

  X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c')(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)

  X=Add()([X,X_shortcut]) # Skip Connection
  X=Activation('relu')(X)

  return X

In [ ]:
def convolutional_block(X,f,filters,stage,block,s=2):
  conv_name_base='res'+str(stage)+ block+'_branch'  #hangi stage'in hangi bloğuna ait
  bn_name_base='bn'+str(stage)+ block+'_branch'
  F1,F2,F3=filters

  X_shortcut=X    # ilk giren X buraya aktarılıyor sonradan kesip eklemek için

  X=Conv2D(filters=F1,kernel_size=(1,1),strides=(s,s),padding='valid',name=conv_name_base+'2a')(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
  X=Activation('relu')(X)

  X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name_base+'2b')(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
  X=Activation('relu')(X)

  X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c')(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)

  # burada da x'i aynı boyuta getirip ekliyoruz.
  X_shortcut=Conv2D(filters=F3,kernel_size=(1,1),strides=(s,s),padding='valid',name=conv_name_base + '1')(X_shortcut)
  X_shortcut=BatchNormalization(axis=3,name=bn_name_base+'1')(X_shortcut)       
  
  X=Add()([X,X_shortcut]) # Skip Connection
  X=Activation('relu')(X)

  return X

In [ ]:
def ResNet50(input_shape=(224,224,3)):

  X_input=Input(input_shape)

  X=ZeroPadding2D((3,3))(X_input)
  X=Conv2D(64,(7,7),strides=(2,2),name='conv1')(X)
  # ((r+2p-f)/2)+1 -> 224+2*3/2 +1
  # 
  X=BatchNormalization(axis=3,name='bn_conv1')(X)
  X=Activation('relu')(X)
  X=MaxPooling2D((3,3),strides=(2,2))(X)

  X=convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
  X=identity_block(X,f=3,filters=[64,64,256],stage=2,block='b')
  X=identity_block(X,f=3,filters=[64,64,256],stage=2,block='c')

  X=convolutional_block(X,f=3,filters=[128,128,512],stage=3,block='a',s=2)
  X=identity_block(X,f=3,filters=[128,128,512],stage=3,block='b')
  X=identity_block(X,f=3,filters=[128,128,512],stage=3,block='c')
  X=identity_block(X,f=3,filters=[128,128,512],stage=3,block='d')

  X=convolutional_block(X,f=3,filters=[256,256,1024],stage=4,block='a',s=2)
  X=identity_block(X,f=3,filters=[256,256,1024],stage=4,block='b')
  X=identity_block(X,f=3,filters=[256,256,1024],stage=4,block='c')
  X=identity_block(X,f=3,filters=[256,256,1024],stage=4,block='d')
  X=identity_block(X,f=3,filters=[256,256,1024],stage=4,block='e')
  X=identity_block(X,f=3,filters=[256,256,1024],stage=4,block='f')

  X=convolutional_block(X,f=3,filters=[512,512,2048],stage=5,block='a',s=2)
  X=identity_block(X,f=3,filters=[512,512,2048],stage=5,block='b')
  X=identity_block(X,f=3,filters=[512,512,2048],stage=5,block='c')  

  # flat yapmak yerine globalavgpooilng yapılmışi
  X=GlobalAveragePooling2D()(X)
  #X=Flatten()(X)
  X=Dense(3,activation='softmax')(X) # biz 3 yazdıkta normalde 1000 olması gerekiyır.

  model=Model(inputs=X_input,outputs=X,name='ResNet50')

  return model

In [ ]:
resnet50=ResNet50()
resnet50.summary()

102967424/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                        

In [ ]:
# kendimiz yazmak yerine hazır getirmek...
from tensorflow.keras.applications import ResNet50

model = ResNet50(include_top=False,input_shape=(224,224,3))

X=Flatten()(model.output)
X=Dense(512,activation='relu')(X)
X=Dense(3,activation='softmax')(X)

model=Model(inputs=model.input,outputs=X,name='ResNet50')

model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           